In [1]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from find_screen_utils import *

#Add Gaze Position to Video

In [6]:
gaze = pd.read_csv('images/tobii/border/border.csv')
vid_sync = pd.read_csv('images/tobii/border/border_video_sync.csv')

In [7]:
vid_sync.head()

,Unnamed: 0,VTS time,VTS validity
0,155690716,0,0
1,155810716,119966,0
2,215677820,59985777,0
3,275662362,119971522,0
4,335649290,179957244,0


In [8]:
gaze['Vid_Time'] = (gaze['index'] - 155690716)/1000000

In [9]:
gaze = gaze[gaze.Vid_Time > 0] # only start tracking eyes once video starts

In [10]:
infile = 'images/tobii/border/border.mp4'
outfile = 'images/tobii/border/border_gaze.m4v'

In [11]:
vid = cv2.VideoCapture(infile)

size = (1920, 1080)
fourcc = cv2.cv.CV_FOURCC(*'mp4v')
out = cv2.VideoWriter()
out.open(outfile, fourcc, vid.get(cv2.cv.CV_CAP_PROP_FPS)*2, size, True)
# note doubled framerate to include all eye tracking coordiantes

i = 0
gaze_val = gaze['gaze position validity'].values
gaze_x = gaze['gaze position x'].values
gaze_y = gaze['gaze position y'].values

while(vid.isOpened()):
    ret, frame = vid.read()
    if ret:
        for _ in range(2):
            frame_cp = frame.copy()
            if gaze_val[i] == 0:
                gp_x = gaze_x[i]
                gp_y = gaze_y[i]
                cv2.circle(frame_cp, (int(1920*gp_x), int(1080*gp_y)), 20, [255,0,0], 2)
            out.write(frame_cp)
            i += 1
    else:
        break


vid.release()
out.release()

#Extract Screen From Gaze Video

In [3]:
infile = 'images/tobii/border/border_gaze_light_lines.m4v'
outfile = 'images/tobii/border/border_gaze_light_lines_screen.m4v'

In [4]:
vid = cv2.VideoCapture(infile)

size = (1280, 720)
fourcc = cv2.cv.CV_FOURCC(*'mp4v')
out = cv2.VideoWriter()
out.open(outfile, fourcc, vid.get(cv2.cv.CV_CAP_PROP_FPS), size, True)

last_good = np.zeros((720, 1280, 3), dtype=np.uint8)  # represents the last good frame

i = 1
while(vid.isOpened()):
    ret, frame = vid.read()
    if ret:
        proc_frame = process_frame_lines(frame, 'light')
        
        if proc_frame is None:  # if screen isn't found, save image of frame
            out.write(last_good)  # write the last good frame to the video
            cv2.imwrite('images/tobii/border/dropped/frame%i.bmp'%(i), frame)
            pass
        else:  # frame is good
            last_good = proc_frame
            out.write(proc_frame)
        i += 1
    else:
        break


vid.release()
out.release()

/usr/local/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
